In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
X_train = pd.read_csv("../data/raw/X_train.csv", sep=",",index_col=0)
Y_train = pd.read_csv("../data/raw/Y_train.csv", sep=",",index_col=0)

In [82]:
# SUPPRESSION DES DOUBLONS EN COLONNE

# D'abord on normalise le texte pour éviter certains pb lors de l'analyse de doublong (sensibilité à la case, encodage, espace en début/fin ...)
def clean_text(text):
    if isinstance(text, str):            # Si le texte est une chaîne de caractères,
        #text = unidecode(text)          # Remplace les accents par des lettres sans accent
        text = text.lower()               # Met le texte en minuscule
        text = text.strip()              # supprime les espaces en début/fin de chaine
        text = ' '.join(text.split())    # supprime les espaces inutiles
        return text
    return text
    
X_train["designation"] = X_train["designation"].map(clean_text)
X_train["description"] = X_train['description'].map(clean_text)


# Avant de concaténer Designation et Description, on vérifie qu'il n'y a pas la même chose dans ces deux colonnes
nb_duplicates = (X_train["designation"] == X_train["description"]).sum()
print("Il y a",nb_duplicates,"lignes où designation et description contiennent les mêmes informations")

# On remplace alors la description par une chaine de caractère vide pour préparer la concaténation
X_train.loc[X_train["designation"] == X_train["description"], "description"] = ""

# On vérifie que le nettoyage a fonctionné
nb_duplicates_ac = (X_train["designation"] == X_train["description"]).sum()
print("Il y a",nb_duplicates_ac,"lignes où designation et description contiennent les mêmes informations après nettoyage")

Il y a 861 lignes où designation et description contiennent les mêmes informations
Il y a 0 lignes où designation et description contiennent les mêmes informations après nettoyage


In [83]:
# CONCATENATION DES COLONNEs DESIGNATION ET DESCRIPTION

print("Nombre de NaN dans la colonne designation:", X_train["designation"].isna().sum())
print("Nombre de NaN dans la colonne description:", X_train["description"].isna().sum(),"\n")

# On remplace les NaN de la colonne "description" par une chaine vide pour que la concaténation fonctionne
# et concatène "designation" et "description" dans une nouvelle colonne full description
# puis on supprime les éventuels espace de fin inutile
X_train["text"] = X_train[["designation", "description"]].fillna("").agg(' '.join, axis=1)
X_train["text"] = X_train["text"].str.strip()

# Suppression des colonnes inutiles
X_train = X_train.drop(columns=['designation','description'])
display(X_train.head(5))

Nombre de NaN dans la colonne designation: 0
Nombre de NaN dans la colonne description: 29800 



,productid,imageid,text
0,3804725264,1263597046,olivia: personalisiertes notizbuch / 150 seite...
1,436067568,1008141237,journal des arts (le) n° 133 du 28/09/2001 - l...
2,201115110,938777978,grand stylet ergonomique bleu gamepad nintendo...
3,50418756,457047496,peluche donald - europe - disneyland 2000 (mar...
4,278535884,1077757786,la guerre des tuques luc a des id&eacute;es de...


In [84]:
# On regarde si on a des doublons
nb_duplicated_text = X_train['text'].duplicated().sum()
print("il existe", nb_duplicated_text, "doublons de texte")

# Suppression des doublons en fonction de la colonne fusionnée
X_train = X_train.drop_duplicates(subset=['text'])
print("suppression des",nb_duplicated_text,"doublons")

il existe 1447 doublons de texte
suppression des 1447 doublons


On remarque en étudiant le contenu du dataset visuellement que celui-ci semble contenir:
* des balise html 
* des entités html (&amp, &nbsp,&lt, &gt ...)

Ces balises et ces entités ne sont pas toujours bien formées (ex: < br >, & nbsp ...)


In [86]:
# traitement des entités html (type &amp, &quot, etc)

import html

X_train["text"] = X_train["text"].apply(html.unescape)

# On va essayer d'identifier les cas les plus fréquents d'entitée html mal formée
df_html = X_train["text"].str.extract(r"(&\s*\w+\s*;)").dropna()
display(df_html.value_counts().head(30))

# Dictionnaire des entités malformées à corriger
html_entities = {
    "& amp;": "&amp;",
    "& nbsp;": "&nbsp;",
    "& lt;": "&lt;",
    "& gt;": "&gt;",
    "& nbsp ;": "&nbsp;",
    "& amp ; ": "&amp;",
    "& gt ;": "&gt;",
    "& quot;": "&quot;"
}

# Remplacer toutes les entités malformées en une seule passe
for incorrect, correct in html_entities.items():
    X_train["text"] = X_train["text"].str.replace(incorrect, correct, regex=False)

# on unescape de nouveau les entités pour gérer les cas corrigés
X_train["text"] = X_train["text"].apply(html.unescape)

0            
& patcher ;      2
& amp ;          1
& hygiénique;    1
& nbsp;          1
& play ;         1
Name: count, dtype: int64

In [87]:
# TRAITEMENT DES BALISES HTML (type <b>, </p>, etc)

# Importation de la bibliothèque "Regular Expression"
import re     

# Fonction qui identifie tout ce qui est délimité par < >
def extraction_balises(texte):
    return re.findall(r"<[^>]+>", str(texte))        

# Création d'une liste d'éléments uniques
balises = set()                                   
X_train["text"].apply(lambda x: balises.update(extraction_balises(x)))

print("Nous avons identifié",len(balises),"balises html uniques présentent dans les descriptions produit")

Nous avons identifié 405 balises html uniques présentent dans les descriptions produit


En jettant consultant la liste des balises identifiées, on observe 
* qu'il n'y a pas que des balises HTML. Il y a également beaucoup de texte délimité par des doubles chevrons, par exemple : << brouillard >>, <>, ou encore <>
* qu'il y a des balises html mal formée < br >, < /p>
* qu'il y a des commentaires html <!-- -->

Si on veut nettoyer efficacement en limitant la perte d'information, je pense qu'il faut d'abord gérer à part les cas comme << brouillard >> en supprimant les doubles chevrons et en conservant le texte.
puis utiliser une regex assez large pour supprimer tout contenu compris entre 2 chevrons.

In [88]:
# suppression des doubles chevrons
X_train["text"] = X_train["text"].str.replace("<<", "").str.replace(">>", "")

# on supprime les balises html quelles soient bien ou mal formées
X_train['text'] = X_train['text'].str.replace(r'<[^>]*>', '', regex=True)

# on vérifie qu'il n'y a plus de balise html
balises2 = set()                                   
X_train["text"].apply(lambda x: balises2.update(extraction_balises(x)))

print("Nous avons identifier",len(balises2),"balises html uniques restant")

Nous avons identifier 0 balises html uniques restant


In [98]:
duplicated = X_train["text"].duplicated().sum()

print("Après traitement du html on se retrouve avec",duplicated,"doublon")

X_train = X_train.drop_duplicates(subset=['text'])

Après traitement du html on se retrouve avec 0 doublon


In [102]:
#sauvegarde du dataframe clean dans un fichier
X_train.to_csv("../data/interim/X_train.csv", index = False)
Y_train.loc[Y_train.index.isin(X_train.index)].to_csv("../data/interim/Y_train.csv", index = False)